In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import pandas as pd
from time import sleep

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.6613.119 Safari/537.36'}

#Necessário para ter as siglas dos times da NBA
times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK',
        'Charlotte Bobcats' : 'CHA',
        'Vancouver Grizzlies' : 'VAN',
        'New Orleans Hornets' : 'NOH',
        'New Orleans/Oklahoma City Hornets' : 'NOK'}

#Intervalos de anos dos dados das temporadas que serão extraidos
anos = list(range(1980,2025))
#anos = [2024]

# Scrapping de todas as temporadas

In [3]:
def scrapping_times(url):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    table_e = soup.find('table', attrs={'id':'confs_standings_E'})
    if table_e == None:
        table_e = soup.find('table', attrs={'id':'divs_standings_E'})
    
    
    th_list = table_e.find('tbody').find_all('th')
    dados_time_e = [x.find('a').get_text().strip() for x in th_list if x.find('a')!=None]

    table_w = soup.find('table', attrs={'id':'confs_standings_W'})
    if table_w == None:
        table_w = soup.find('table', attrs={'id':'divs_standings_W'})
        
    th_list = table_w.find('tbody').find_all('th')

        
    th_list = table_w.find('tbody').find_all('th')

    dados_time_w = [x.find('a').get_text().strip() for x in th_list if x.find('a')!=None]
    
    return dados_time_e + dados_time_w

In [4]:
def scrapping_dados_jogos(url, time_casa, season):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    except Exception as e:
        print(e)

    dados_jogos = []

    table_games = soup.find('table', attrs={'id':'games'})
    tr_keys = table_games.find('thead').find_all('th')
    keys = [x.get_text().strip() for x in tr_keys]

    tbody_games = table_games.find('tbody')
    tr_list = tbody_games.find_all('tr')

    for tr_inst in tr_list:
        if(tr_inst.get('class') == None):
            dados_jogo = {}
            dados_jogo['g'] = tr_inst.find('th').get_text().strip()
            dados_jogo['season'] = season
            dados_jogo['date_game'] =  tr_inst.find('td', attrs={'data-stat':'date_game'}).find('a').get_text().strip()
            if  tr_inst.find('td', attrs={'data-stat':'game_start_time'}) != None :
                dados_jogo['game_start_time'] = tr_inst.find('td', attrs={'data-stat':'game_start_time'}).get_text().strip()
            else:
                dados_jogo['game_start_time'] = None
            dados_jogo['link'] = 'https://www.basketball-reference.com' + tr_inst.find('td', attrs={'data-stat':'box_score_text'}).find('a').get('href')
            dados_jogo['team_1'] = time_casa
            team_2 = tr_inst.find('td', attrs={'data-stat':'opp_name'}).find('a').get_text().strip()
            dados_jogo['team_2'] = times[team_2]
            dados_jogo['home'] = 0 if tr_inst.find('td', attrs={'data-stat':'game_location'}).get_text().strip() != '@' else 1
            dados_jogo['game_result'] = tr_inst.find('td', attrs={'data-stat':'game_result'}).get_text().strip()
            dados_jogo['pts'] = tr_inst.find('td', attrs={'data-stat':'pts'}).get_text().strip()
            dados_jogo['opp_pts'] = tr_inst.find('td', attrs={'data-stat':'opp_pts'}).get_text().strip()
            dados_jogo['wins'] = tr_inst.find('td', attrs={'data-stat':'wins'}).get_text().strip()
            dados_jogo['losses'] = tr_inst.find('td', attrs={'data-stat':'losses'}).get_text().strip()
            dados_jogo['game_streak'] = tr_inst.find('td', attrs={'data-stat':'game_streak'}).get_text().strip()
            dados_jogo['game_remarks'] = tr_inst.find('td', attrs={'data-stat':'game_remarks'}).get_text().strip()

            dados_jogos.append(dados_jogo)
            
            
    return dados_jogos

In [4]:
dados = []
for index_ano, ano in enumerate(anos):
    # Capturando os times daquela temporada (Necessário, pois franquias podem ser vendidas)
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    
    try:
        times_participantes = scrapping_times(url_dados_time)
    except:
        print("Erro na captura de times da temporada {}".format(ano))
    
    print('Capturados os times da temporada {}'.format(ano))
    
    siglas_times_participantes = [times[x] for x in times_participantes]
    
    for index_sigla, sigla in enumerate(siglas_times_participantes):
        url_dados_jogos = f'https://www.basketball-reference.com/teams/{sigla}/{ano}_games.html'
        
        try:
            dados_time = scrapping_dados_jogos(url_dados_jogos, sigla, ano)
        except:
            print("Erro no time {} na temporada {}".format(sigla, ano))
        
        print('Capturado os jogos do time {} (index {}) da temporada {} (index {})'.format(sigla, index_sigla, ano, index_ano))
        sleep(2.5)
        dados.append(dados_time)

    print('Capturados os jogos da temporada {} (index {})'.format(ano, index_ano))

Capturados os times da temporada 1980
Capturado os jogos do time BOS (index 0) da temporada 1980 (index 0)
Capturado os jogos do time PHI (index 1) da temporada 1980 (index 0)
Capturado os jogos do time WSB (index 2) da temporada 1980 (index 0)
Capturado os jogos do time NYK (index 3) da temporada 1980 (index 0)
Capturado os jogos do time NJN (index 4) da temporada 1980 (index 0)
Capturado os jogos do time ATL (index 5) da temporada 1980 (index 0)
Capturado os jogos do time HOU (index 6) da temporada 1980 (index 0)
Capturado os jogos do time SAS (index 7) da temporada 1980 (index 0)
Capturado os jogos do time IND (index 8) da temporada 1980 (index 0)
Capturado os jogos do time CLE (index 9) da temporada 1980 (index 0)
Capturado os jogos do time DET (index 10) da temporada 1980 (index 0)
Capturado os jogos do time MIL (index 11) da temporada 1980 (index 0)
Capturado os jogos do time KCK (index 12) da temporada 1980 (index 0)
Capturado os jogos do time DEN (index 13) da temporada 1980 (i

In [6]:
dados_jogos = []
for dado in dados:
    for jogo_dado in dado:
        dados_jogos.append(jogo_dado)
dataframe = pd.DataFrame(dados_jogos)

dataframe.head()

,g,season,date_game,game_start_time,link,team_1,team_2,home,game_result,pts,opp_pts,wins,losses,game_streak,game_remarks
0,1,1980,"Fri, Oct 12, 1979",None,https://www.basketball-reference.com/boxscores...,BOS,HOU,0,W,114,106,1,0,W 1,
1,2,1980,"Sat, Oct 13, 1979",None,https://www.basketball-reference.com/boxscores...,BOS,CLE,1,W,139,117,2,0,W 2,
2,3,1980,"Wed, Oct 17, 1979",None,https://www.basketball-reference.com/boxscores...,BOS,CLE,0,W,127,108,3,0,W 3,
3,4,1980,"Fri, Oct 19, 1979",None,https://www.basketball-reference.com/boxscores...,BOS,WSB,0,W,130,93,4,0,W 4,
4,5,1980,"Sat, Oct 20, 1979",None,https://www.basketball-reference.com/boxscores...,BOS,IND,1,L,128,131,4,1,L 1,


In [17]:
# Retirando os valores com home=1 para que o team_2 seja sempre o time visitante
dataframe = dataframe.drop(dataframe[dataframe['home'] == 1].index)
dataframe.to_csv('./dados_jogos_pt2.csv', index=False)

In [3]:
def scrapping_jogo(url, dados_partida):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

    except Exception as e:
        print(e)

    # Capturando os dados da partida do time 1
    table_team1 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_1'])})
    if table_team1 == None and dados_partida['team_1'] == 'CHO':
        dados_partida['team_1'] = 'CHH'
        table_team1 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_1'])})
    
    th_list_team1 = table_team1.find('thead').find_all('tr')[1].find_all('th')

    keys_basic =  [x.get_text().strip().lower() for x in th_list_team1]

    td_basic_team1 = table_team1.find('tfoot').find('tr').find_all('td')

    i = 1
    for td_inst in td_basic_team1[:-2]:
        key = 'team_1_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()
    

    table_advanced_team1 = soup.find('table', attrs={'id':'box-{}-game-advanced'.format(dados_partida['team_1'])})
    
    th_list_team1 = table_advanced_team1.find('thead').find_all('tr')[1].find_all('th')

    keys_advanced =  [x.get_text().strip().lower() for x in th_list_team1]

    td_advanced_team1 = table_advanced_team1.find('tfoot').find('tr').find_all('td')

    i = 1
    for td_inst in td_advanced_team1[:-1]:
        key = 'team_1_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()

    # Capturando os dados do time 2
    print("Time 2:", dados_partida['team_2'])
    table_team2 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_2'])})
    if table_team2 == None and dados_partida['team_2'] == 'CHO':
        dados_partida['team_2'] = 'CHH'
        table_team2 = soup.find('table', attrs={'id':'box-{}-game-basic'.format(dados_partida['team_2'])})
    #th_list_team2 = table_team2.find('thead').find_all('tr')[1].find_all('th')
    td_basic_team2 = table_team2.find('tfoot').find('tr').find_all('td')

    for td_inst in td_basic_team2[:-2]:
        key = 'team_2_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()

    table_advanced_team2 = soup.find('table', attrs={'id':'box-{}-game-advanced'.format(dados_partida['team_2'])})
    #th_list_team2 = table_advanced_team2.find('thead').find_all('tr')[1].find_all('th')

    td_advanced_team2 = table_advanced_team2.find('tfoot').find('tr').find_all('td')
    #print("Teste:", td_advanced_team2)
    for td_inst in td_advanced_team2[:-1]:
        key = 'team_2_' + td_inst.get('data-stat').strip()
        dados_partida[key] = td_inst.get_text().strip()
    

In [36]:
dataframe = pd.read_csv('./dados_jogos_pt2.csv')
dataframe['season'] = dataframe['season'].astype(int)
dataframe = dataframe[dataframe['season']>2000]
dataframe.head()

,g,season,date_game,game_start_time,link,team_1,team_2,home,game_result,pts,opp_pts,wins,losses,game_streak,game_remarks
21594,2,2001,"Wed, Nov 1, 2000",7:00p,https://www.basketball-reference.com/boxscores...,PHI,TOR,0,W,104,98,2,0,W 2,NaN
21595,5,2001,"Wed, Nov 8, 2000",7:00p,https://www.basketball-reference.com/boxscores...,PHI,DET,0,W,103,94,5,0,W 5,NaN
21596,7,2001,"Sat, Nov 11, 2000",8:00p,https://www.basketball-reference.com/boxscores...,PHI,BOS,0,W,85,83,7,0,W 7,NaN
21597,8,2001,"Wed, Nov 15, 2000",7:00p,https://www.basketball-reference.com/boxscores...,PHI,CLE,0,W,107,98,8,0,W 8,NaN
21598,9,2001,"Fri, Nov 17, 2000",7:00p,https://www.basketball-reference.com/boxscores...,PHI,MIA,0,W,94,73,9,0,W 9,NaN


In [37]:
colunas = list(dataframe.columns)
dados = []
for index, row in dataframe.iterrows():
    dados_jogo = {}
    for coluna in colunas:
        dados_jogo[coluna] = row[coluna]
    dados.append(dados_jogo)

In [38]:
dados[:10]

[{'g': 2,
  'season': 2001,
  'date_game': 'Wed, Nov 1, 2000',
  'game_start_time': '7:00p',
  'link': 'https://www.basketball-reference.com/boxscores/200011010PHI.html',
  'team_1': 'PHI',
  'team_2': 'TOR',
  'home': 0,
  'game_result': 'W',
  'pts': 104,
  'opp_pts': 98,
  'wins': 2,
  'losses': 0,
  'game_streak': 'W 2',
  'game_remarks': nan},
 {'g': 5,
  'season': 2001,
  'date_game': 'Wed, Nov 8, 2000',
  'game_start_time': '7:00p',
  'link': 'https://www.basketball-reference.com/boxscores/200011080PHI.html',
  'team_1': 'PHI',
  'team_2': 'DET',
  'home': 0,
  'game_result': 'W',
  'pts': 103,
  'opp_pts': 94,
  'wins': 5,
  'losses': 0,
  'game_streak': 'W 5',
  'game_remarks': nan},
 {'g': 7,
  'season': 2001,
  'date_game': 'Sat, Nov 11, 2000',
  'game_start_time': '8:00p',
  'link': 'https://www.basketball-reference.com/boxscores/200011110PHI.html',
  'team_1': 'PHI',
  'team_2': 'BOS',
  'home': 0,
  'game_result': 'W',
  'pts': 85,
  'opp_pts': 83,
  'wins': 7,
  'losses'

In [5]:
for index_dados_jogo in range(27790, len(dados)):
    dados_jogo_time = dados[index_dados_jogo]
    url_jogo = dados_jogo_time['link']
    print(url_jogo)
    scrapping_jogo(url_jogo, dados_jogo_time)
    print('Capturado dados do jogo (index {})'. format(index_dados_jogo))
    sleep(2.5)

https://www.basketball-reference.com/boxscores/202310300IND.html
Time 2: CHI
Capturado dados do jogo (index 27790)
https://www.basketball-reference.com/boxscores/202311030IND.html
Time 2: CLE
Capturado dados do jogo (index 27791)
https://www.basketball-reference.com/boxscores/202311040IND.html
Time 2: CHO
Capturado dados do jogo (index 27792)
https://www.basketball-reference.com/boxscores/202311060IND.html
Time 2: SAS
Capturado dados do jogo (index 27793)
https://www.basketball-reference.com/boxscores/202311080IND.html
Time 2: UTA
Capturado dados do jogo (index 27794)
https://www.basketball-reference.com/boxscores/202311090IND.html
Time 2: MIL
Capturado dados do jogo (index 27795)
https://www.basketball-reference.com/boxscores/202311190IND.html
Time 2: ORL
Capturado dados do jogo (index 27796)
https://www.basketball-reference.com/boxscores/202311220IND.html
Time 2: TOR
Capturado dados do jogo (index 27797)
https://www.basketball-reference.com/boxscores/202311240IND.html
Time 2: DET
Cap

In [6]:
import pickle

with open('dados.pkl', 'wb') as f:
    dados = pickle.dump(dados, f)    

In [9]:
len(dados)

28819